In [10]:
ENV["GKS_ENCODING"] = "utf-8"
using Plots
include("src/ladderDGATools.jl")
include("src/ladderDGA_Julia.jl");

Inputs Read. Starting Computation                                          
Calculating bubble:   1.260855 seconds (15.17 M allocations: 486.534 MiB, 22.68% gc time)
Calculating naiive χ and γ in the charge channel:   1.714937 seconds (1.06 M allocations: 274.367 MiB, 3.69% gc time)
Calculating naiive χ and γ in the spin channel:   1.095997 seconds (51.87 k allocations: 223.626 MiB, 2.87% gc time)
Calculating λ correction in the charge channel: aa
  0.447439 seconds (1.30 M allocations: 63.945 MiB, 7.37% gc time)
Calculating λ correction in the spin channel: aa
  0.004244 seconds (131 allocations: 1.347 MiB)
Calculating naiive Σ ladder:   0.313267 seconds (248.28 k allocations: 13.545 MiB)
Calculating χ and γ in the charge channel:   1.550348 seconds (1.01 M allocations: 281.373 MiB, 5.03% gc time)
Calculating χ and γ in the spin channel:   1.571588 seconds (58.26 k allocations: 233.116 MiB, 2.32% gc time)
TODO: replace chisp by chisp_lambda


In [24]:
include("src/ladderDGATools.jl");
include("src/lambdaCorrection.jl");

In [13]:
@time sol = find_λ(G0, χch_impr, χsp_impr, trilexch_impr, trilexsp_impr, bubble, Σ_loc, FUpDo,
        qMultiplicity, qGrid, kGrid, modelParams, simParams, [0.0, 0.0])

NaN
741.270858 seconds (8.72 M allocations: 5.253 GiB, 0.14% gc time)


 * Status: failure (reached maximum number of iterations) (line search failed)

 * Candidate solution
    Minimizer: [0.00e+00, 0.00e+00]
    Minimum:   NaN

 * Found with
    Algorithm:     Nelder-Mead
    Initial Point: [0.00e+00, 0.00e+00]

 * Convergence measures
    √(Σ(yᵢ-ȳ)²)/n ≰ 1.0e-08

 * Work counters
    Seconds run:   739  (vs limit Inf)
    Iterations:    1000
    f(x) calls:    4005


In [17]:
Optim.minimizer(sol)

2-element Array{Float64,1}:
 -0.046019482048729236
  0.005766004636100292

In [ ]:
@time sol2 = find_λ4(G0, χch_impr, χsp_impr, trilexch_impr, trilexsp_impr, bubble, Σ_loc, FUpDo,
        qMultiplicity, qGrid, kGrid, modelParams, simParams, 
        [1.0,1.0],BFGS())

142477.62546982893


In [ ]:
Optim.minimizer(sol2)

In [2]:
const ll = [(x1,x2) for x1 in range(-5,stop=5,length=2) for x2 in range(-5,stop=5,length=2)]
res = SharedArray{Float64}(lenght(ll))
@sync @distributed for lli in 1:length(ll)
    res[lli] = eval_f(ll[lli], G0, χch_impr, χsp_impr, trilexch_impr, trilexsp_impr, bubble, Σ_loc, FUpDo,
        qMultiplicity, qGrid, kGrid, modelParams, simParams)
end

In [ ]:
n = length(ll)
@elapsed while n > 0 # print out results
    job_id, exec_time = take!(results)
    println("$job_id finished in $(round(exec_time; digits=2)) seconds")
    global n = n - 1
end